In [65]:
import pandas as pd
from uuid import uuid4
import datetime
import dateutil
import time
from random import random, randint

from tribeMaster.hygiene.present.refData.conform_refData import ReferenceDataServiceFactory
from tribeMaster.hygiene.present.refData.conform_refData_pb2 import NWoWRole, Location, TribeMember, Squad

In [26]:
def _getPI(path_data):
    refServicer = ReferenceDataServiceFactory.getReferenceDataService(path_data)
    pis = refServicer.getSprints(None,None)
    for pi in pis.pis:
        yield pi
        
def _tzTransfer(ts):
    from_zone = dateutil.tz.gettz('UTC')
    to_zone = dateutil.tz.gettz('Australia/Melbourne')
    ts = datetime.datetime.fromtimestamp(ts.seconds).replace(tzinfo=from_zone).astimezone(to_zone)
    return f'{ts.year}-{ts.month}-{ts.day}'

def _getNowTs():
    return time.mktime(datetime.datetime.now().timetuple())


In [27]:
PATH_DATA = './'

In [28]:
pis = _getPI(PATH_DATA)
now = _getNowTs()

In [29]:
pi_ls = []
sprint_ls = []
for pi in pis: 
    pi_id = str(uuid4())
    pi_ls.append({
        'piID':pi_id,
        'piNumber': pi.seqNo, 
        'startDate': _tzTransfer(pi.start), 
        'endDate':_tzTransfer(pi.end),
        'isCurrent': pi.start.seconds<now and pi.end.seconds>now,
    })
    for sprint in pi.sprints:
        sprint_id = str(uuid4())
        sprint_ls.append({
            'sprintID':sprint_id, 
            'sprintNumber':sprint.seqNo, 
            'isCurrent':sprint.start.seconds<now and sprint.end.seconds>now, 
            'startDate':_tzTransfer(sprint.start), 
            'endDate':_tzTransfer(sprint.end),
            'piID': pi_id
        })

In [31]:
pi_df = pd.DataFrame(pi_ls)

In [32]:
sprint_df = pd.DataFrame(sprint_ls)

In [44]:
pi_df.to_csv('pi.csv', index=False)

In [45]:
sprint_df.to_csv('sprint.csv', index=False)

In [38]:
squad_df = pd.read_csv('squad.csv')

In [41]:
graphID = str(uuid4())
graph = [{
    'graphID': graphID,
    'name': 'test graph',
    'version': '0.0.0',
    'description': 'test description',
}]
graph_df = pd.DataFrame(graph)

In [46]:
graph_df.to_csv('graph.csv', index=False)

In [53]:
squad_df.head()

,squadID,name,mission,focusID,health
0,f8c7d5fe-9384-439c-8891-4c40361687e6,SquadOne,Mission One,60bae18b-3803-4f06-ab75-f686521e28a8,0.80
1,e90466cd-1463-417f-8242-f6070f097de2,SquadTwo,Mission Two,cd150409-95fd-409d-be56-6c2a24005f5a,0.75
2,ceb41caa-7e8d-4bea-8fd1-f07d47fffae7,SquadThree,Mission Three,1d4ef9d3-338b-44fd-80bf-034d2a42bf70,0.75
3,77980081-2601-4347-b338-ac848766cf46,SquadFour,Mission Four,60bae18b-3803-4f06-ab75-f686521e28a8,0.80
4,55bfc995-8d25-4cba-8ea2-294dc6eaeb62,SquadFive,Mission Five,cd150409-95fd-409d-be56-6c2a24005f5a,0.40


In [115]:
gs_ls = []
for i,row in squad_df.iterrows():
    graph_squad = {
        'graphID': graphID,
        'squadID': row['squadID'],
        'upToDate': True, 
        'analyseDate': '2019-07-01',
        'piNumber': 5,
        'values': str([round(random(),2) for _ in range(8)]).replace('[','').replace(']',''),
        'valuesMask': str([True for _ in range(8)]).replace('[','').replace(']',''),
        'coordinates': str(['sprint1', 'sprint2', 'code_impact', 'code_contrib', 'person1', 'person2']).replace('[','').replace(']',''),
        'valuesShape': str([2,2,2]).replace('[','').replace(']','')
    }
    gs_ls.append(graph_squad)

In [116]:
gs_df = pd.DataFrame(gs_ls)

In [117]:
gs_df.to_csv('squad_graph.csv', index=False)

In [118]:
gs_df.head()

,analyseDate,coordinates,graphID,piNumber,squadID,upToDate,values,valuesMask,valuesShape
0,2019-07-01,"'sprint1', 'sprint2', 'code_impact', 'code_con...",168deb2e-e4f9-4f3b-94d5-705916908ffc,5,f8c7d5fe-9384-439c-8891-4c40361687e6,True,"0.89, 0.84, 0.02, 0.2, 0.36, 0.15, 0.48, 0.97","True, True, True, True, True, True, True, True","2, 2, 2"
1,2019-07-01,"'sprint1', 'sprint2', 'code_impact', 'code_con...",168deb2e-e4f9-4f3b-94d5-705916908ffc,5,e90466cd-1463-417f-8242-f6070f097de2,True,"0.02, 0.36, 0.96, 0.85, 0.81, 0.06, 0.97, 0.63","True, True, True, True, True, True, True, True","2, 2, 2"
2,2019-07-01,"'sprint1', 'sprint2', 'code_impact', 'code_con...",168deb2e-e4f9-4f3b-94d5-705916908ffc,5,ceb41caa-7e8d-4bea-8fd1-f07d47fffae7,True,"0.9, 0.22, 0.27, 0.63, 0.31, 0.45, 1.0, 0.23","True, True, True, True, True, True, True, True","2, 2, 2"
3,2019-07-01,"'sprint1', 'sprint2', 'code_impact', 'code_con...",168deb2e-e4f9-4f3b-94d5-705916908ffc,5,77980081-2601-4347-b338-ac848766cf46,True,"0.2, 0.31, 0.44, 0.91, 0.04, 0.48, 0.05, 0.03","True, True, True, True, True, True, True, True","2, 2, 2"
4,2019-07-01,"'sprint1', 'sprint2', 'code_impact', 'code_con...",168deb2e-e4f9-4f3b-94d5-705916908ffc,5,55bfc995-8d25-4cba-8ea2-294dc6eaeb62,True,"0.21, 0.45, 0.7, 0.39, 0.31, 0.32, 0.73, 0.65","True, True, True, True, True, True, True, True","2, 2, 2"


In [64]:
# jira data
STATUS = [
    'FUNNEL',
    'WITHTEAM',
    'BACKLOG',
    'TODO',
    'INPROGRESS',
    'INREVIEW',
    'INTESTING',
    'COMPLETED',
    'UNKNOWN'
]

In [69]:
features = []
for i in range(3):
    features.append({
        'featureID': str(uuid4()),
        'jiraID': f'BASE{i}',
        'status': STATUS[randint(0,9)],
    })
featureDF = pd.DataFrame(features)
featureDF.head()

,featureID,jiraID,status
0,4c17a163-e48f-4e3b-9be3-8425211cd23b,BASE0,COMPLETED
1,a6ef1d84-55d0-47b7-81df-8f99c54847ee,BASE1,INREVIEW
2,faf1dc0d-4043-4fe3-b1ea-95300452459e,BASE2,INTESTING


In [72]:
pi_df.head()

,endDate,isCurrent,piID,piNumber,startDate
0,2019-2-25,False,301c97b3-57f8-4397-8ff9-2fc9dc25423e,4,2018-12-18
1,2019-6-3,False,cb41a4c8-38a4-4257-8a68-7ac4ba2db62d,5,2019-2-26
2,2019-9-9,True,f4e8c2f7-4193-414a-96b2-25ccc86cb64b,6,2019-6-4
3,2019-12-16,False,03fd9315-76f9-4393-a0aa-fd5e562f2798,7,2019-9-10


In [74]:
list(pi_df['piID'])

['301c97b3-57f8-4397-8ff9-2fc9dc25423e',
 'cb41a4c8-38a4-4257-8a68-7ac4ba2db62d',
 'f4e8c2f7-4193-414a-96b2-25ccc86cb64b',
 '03fd9315-76f9-4393-a0aa-fd5e562f2798']

In [70]:
featureDF.to_csv('feature.csv',index=False)

In [80]:
def _getRandID(series):
    return list(series)[randint(0,len(series))]

In [82]:
epics = []
for i in range(6):
    epics.append({
        'epicID': str(uuid4()),
        'jiraID': f'PTS-{i}',
        'status': STATUS[randint(0,len(STATUS))],
        'scheduledPI': _getRandID(pi_df['piID']),
        'executedPI': _getRandID(pi_df['piID']),
        'featureID': _getRandID(featureDF['featureID']),
        'squadID': _getRandID(squad_df['squadID']),
    })

In [83]:
epics_df = pd.DataFrame(epics)

In [84]:
epics_df.head()

,epicID,executedPI,featureID,jiraID,scheduledPI,squadID,status
0,134489dc-125e-45cd-ba95-9fa9f051130f,cb41a4c8-38a4-4257-8a68-7ac4ba2db62d,faf1dc0d-4043-4fe3-b1ea-95300452459e,PTS-0,cb41a4c8-38a4-4257-8a68-7ac4ba2db62d,55bfc995-8d25-4cba-8ea2-294dc6eaeb62,WITHTEAM
1,087b248e-165f-46c7-af77-310b7f3e797b,f4e8c2f7-4193-414a-96b2-25ccc86cb64b,a6ef1d84-55d0-47b7-81df-8f99c54847ee,PTS-1,f4e8c2f7-4193-414a-96b2-25ccc86cb64b,55bfc995-8d25-4cba-8ea2-294dc6eaeb62,WITHTEAM
2,cb85f9b2-7207-4f05-b488-751485e799b8,f4e8c2f7-4193-414a-96b2-25ccc86cb64b,a6ef1d84-55d0-47b7-81df-8f99c54847ee,PTS-2,cb41a4c8-38a4-4257-8a68-7ac4ba2db62d,ceb41caa-7e8d-4bea-8fd1-f07d47fffae7,INREVIEW
3,74dfdeef-3597-4cab-8bbd-b257cc1cc494,03fd9315-76f9-4393-a0aa-fd5e562f2798,faf1dc0d-4043-4fe3-b1ea-95300452459e,PTS-3,301c97b3-57f8-4397-8ff9-2fc9dc25423e,e3899511-328d-4aac-a19f-bab14b2e2aaa,COMPLETED
4,2b7f185c-edb2-4a48-b8b2-4067ee54f8fb,301c97b3-57f8-4397-8ff9-2fc9dc25423e,faf1dc0d-4043-4fe3-b1ea-95300452459e,PTS-4,cb41a4c8-38a4-4257-8a68-7ac4ba2db62d,e90466cd-1463-417f-8242-f6070f097de2,UNKNOWN


In [85]:
epics_df.to_csv('epic.csv', index=False)

In [86]:
epics_df['dependedOn'] = ['134489dc-125e-45cd-ba95-9fa9f051130f' for _ in range(len(epics_df))]

In [87]:
epics_df['dependedBy'] = ['cb85f9b2-7207-4f05-b488-751485e799b8' for _ in range(len(epics_df))]

In [88]:
epics_df.to_csv('epic.csv', index=False)

In [89]:
StoryType = [
    'STORY',
    'BUG',
    'TEST',
    'TASK',
    'SUBTASK',
    'UNKNOWN',
]

In [95]:
StoryLinkType = ['T1', 'T2', 'T3']

In [93]:
person_df = pd.read_csv('person.csv')

In [94]:
person_df.head()

,personID,lanID,firstName,middleName,lastName,email,phone,isChapterLead,chapterID,locationID,organisationID,skills
0,a45a65c5-e1d9-40e5-92c8-57cca66192bf,lanid1,Evelyn,NaN,Grant,lanid1@anz.com,673-2662-12,True,b95d3166-a3d7-48be-ab25-6a6036231943,8fca03b0-1d78-4254-a25f-26b8ac5dd94a,bfb8e1dc-61d8-4e3b-b18e-85836431a63a,981b718b-5df7-4a97-88da-6afdf0f78497;845834df-...
1,cb5669b8-db3c-4ee7-9ce8-04a5bbf0e388,lanid2,Roman,NaN,Montgomery,lanid2@anz.com,118-0011-31,False,b95d3166-a3d7-48be-ab25-6a6036231943,8fca03b0-1d78-4254-a25f-26b8ac5dd94a,bfb8e1dc-61d8-4e3b-b18e-85836431a63a,fbbda3cc-5408-491c-ad5f-656c9becff5e;167f7cde-...
2,aa60b118-d8cb-4d4a-9dd2-72550f6cdc81,lanid3,Eddy,NaN,Walker,lanid3@anz.com,313-4538-39,False,b95d3166-a3d7-48be-ab25-6a6036231943,8fca03b0-1d78-4254-a25f-26b8ac5dd94a,bfb8e1dc-61d8-4e3b-b18e-85836431a63a,fbbda3cc-5408-491c-ad5f-656c9becff5e;11ddca64-...
3,0e5aa0c1-3424-4099-b5f3-4d386f056245,lanid4,Heather,NaN,Richards,lanid4@anz.com,952-5421-78,False,b95d3166-a3d7-48be-ab25-6a6036231943,8fca03b0-1d78-4254-a25f-26b8ac5dd94a,bfb8e1dc-61d8-4e3b-b18e-85836431a63a,845834df-54d2-4e93-ba39-10a153d731f9;c0a3cc31-...
4,46c6162c-77a1-447d-95e8-d86018fb532c,lanid5,Brad,NaN,Morrison,lanid5@anz.com,849-4389-38,False,b95d3166-a3d7-48be-ab25-6a6036231943,8fca03b0-1d78-4254-a25f-26b8ac5dd94a,bfb8e1dc-61d8-4e3b-b18e-85836431a63a,981b718b-5df7-4a97-88da-6afdf0f78497;069ecde7-...


In [102]:
stories = []
for i in range(10):
    stories.append({
        'storyID':str(uuid4()),
        'jiraID': f'PTS-{i}',
        'status': _getRandID(STATUS),
        'storyType': _getRandID(StoryType),
        'scheduledSprint': _getRandID(sprint_df['sprintID']),
        'executedSprint': _getRandID(sprint_df['sprintID']),
        'epicID': _getRandID(epics_df['epicID']),
        'assigneeID': _getRandID(person_df['personID']),
        'linkType': _getRandID(StoryLinkType)
    })
story_df = pd.DataFrame(stories)


In [103]:
story_df['linkID'] = [_getRandID(story_df['storyID']) for _ in range(len(story_df))]

In [107]:
story_df.to_csv('story.csv', index=False)
story_df.head()

,assigneeID,epicID,executedSprint,jiraID,linkType,scheduledSprint,status,storyID,storyType,linkID
0,6692d2c9-6833-49ca-bbec-f927f997da7d,4bec7f38-06be-4c6a-a223-88bda1c141aa,9de47589-ba72-4f06-9964-bbd5681a7a29,PTS-0,T1,9de47589-ba72-4f06-9964-bbd5681a7a29,INPROGRESS,1230fa76-b332-4bfa-b476-ab2fa98acd95,SUBTASK,1230fa76-b332-4bfa-b476-ab2fa98acd95
1,0e5aa0c1-3424-4099-b5f3-4d386f056245,2b7f185c-edb2-4a48-b8b2-4067ee54f8fb,e56adc95-8cb0-4850-8f25-4a1c1da18a6c,PTS-1,T2,22cbcdfe-fd8d-4fc8-afbc-9a5078c55be1,UNKNOWN,b8b64715-ee31-40a5-a5a0-c4f559d4421a,STORY,65e30aa0-ef49-4ff3-b067-b8efb1554396
2,a57d068b-0e0b-416c-862b-dc1cbc83d73c,74dfdeef-3597-4cab-8bbd-b257cc1cc494,9de47589-ba72-4f06-9964-bbd5681a7a29,PTS-2,T2,3d92e917-199a-49af-be92-48e760456394,WITHTEAM,937759ca-e631-44c5-86ce-35630072985e,BUG,1230fa76-b332-4bfa-b476-ab2fa98acd95
3,e4cec851-2b2e-4c28-b94b-bbf455fc8f27,134489dc-125e-45cd-ba95-9fa9f051130f,1dfe8c14-64e5-40a0-8eec-272db70b321d,PTS-3,T2,22cbcdfe-fd8d-4fc8-afbc-9a5078c55be1,INREVIEW,084667bc-a348-45f5-8fde-f1a055fa3a0a,UNKNOWN,1d35dae1-67cf-4066-a9e7-c8a6f7b0b3a0
4,8e9cd1e5-122a-4bab-99c7-9e60bb45456a,2b7f185c-edb2-4a48-b8b2-4067ee54f8fb,72df1f39-b4af-4d7d-a3cd-aa56fb48cab0,PTS-4,T2,3d92e917-199a-49af-be92-48e760456394,BACKLOG,9a1a4a35-13e2-40f5-a920-6a6a7980172a,TEST,65e30aa0-ef49-4ff3-b067-b8efb1554396


In [106]:
from py2neo import cypher